In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from local_utils import detect_lp
from os.path import splitext, basename
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

import glob
import tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)
        
wpod_net_path = "wpod-net.json"
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [3]:
def preprocess_image(image_path,resize=False):
    img = cv2.imread(image_path) # reads the parsing image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # converts it to RGB
    img = img / 255 # normalizes the image data to 0–1 range
    if resize:
# set resize = True to resize all images to same dimension of (width = 224, height = 224)
#for visualizing purpos
        img = cv2.resize(img, (224,224)) 
    return img

In [42]:
# Create a list of image paths 
image_paths = glob.glob("plate_numbers/*.jpg")
print("Found %i images..."%(len(image_paths)))

# Visualize data in subplot 
fig = plt.figure(figsize=(8,8))
cols = 5
rows = 4
fig_list = []
for i in range(cols*rows):
    fig_list.append(fig.add_subplot(rows,cols,i+1))
    title = splitext(basename(image_paths[i]))[0]
    fig_list[-1].set_title(title)
    img = preprocess_image(image_paths[i],True)
    plt.axis(False)
    plt.imshow(img)

plt.tight_layout(True)
plt.show()

Found 44 images...


![title](resources/cochecitos.png)

In [5]:
def get_plate(image_path, Dmax=3500, Dmin=150):
    vehicle = preprocess_image(image_path)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, vehicle, bound_dim, lp_threshold=0.5)
    return LpImg, cor

In [29]:
# Obtain plate image and its coordinates from an image
test_image = image_paths[19]
LpImg,cor = get_plate(test_image)
print("Detect %i plate(s) in"%len(LpImg),splitext(basename(test_image))[0])
print("Coordinate of plate(s) in image: \n", cor)

Detect 1 plate(s) in OIP (27)
Coordinate of plate(s) in image: 
 [array([[ 60.39824351, 208.09143599, 203.89332867,  56.20013619],
       [ 70.25989705, 101.92575911, 149.44307911, 117.77721705],
       [  1.        ,   1.        ,   1.        ,   1.        ]])]


In [45]:
# Visualize our result
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.axis(False)
plt.imshow(preprocess_image(test_image))
plt.subplot(1,2,2)
plt.axis(False)
plt.imshow(LpImg[0])

![title](resources/big_and_roi.png)

In [47]:
def draw_box(image_path, cor, thickness=3): 
    pts=[]  
    x_coordinates=cor[0][0]
    y_coordinates=cor[0][1]
    # store the top-left, top-right, bottom-left, bottom-right 
    # of the plate license respectively
    for i in range(4):
        pts.append([int(x_coordinates[i]),int(y_coordinates[i])])
    
    pts = np.array(pts, np.int32)
    pts = pts.reshape((-1,1,2))
    vehicle_image = preprocess_image(image_path)
    
    cv2.polylines(vehicle_image,[pts],True,(0,255,0),thickness)
    return vehicle_image

plt.figure(figsize=(8,8))
plt.axis(False)
plt.imshow(draw_box(test_image,cor))

![title](resources/ROI_verde.png)

In [49]:
# Viualize all obtained plate images 
fig = plt.figure(figsize=(12,6))
cols = 5
rows = 4
fig_list = []

for i in range(cols*rows):
    try:
        fig_list.append(fig.add_subplot(rows,cols,i+1))
        title = splitext(basename(image_paths[i]))[0]
        fig_list[-1].set_title(title)
        LpImg,_ = get_plate(image_paths[i])
        plt.axis(False)
        plt.imshow(LpImg[0])
    except:
        pass

plt.tight_layout(True)
plt.show()

![title](resources/plaquitas.png)

In [13]:
if (len(LpImg)): #check if there is at least one license image
    # Scales, calculates absolute values, and converts the result to 8-bit.
    plate_image = cv2.convertScaleAbs(LpImg[0], alpha=(255.0))
    
    # convert to grayscale and blur the image
    gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
    # Blur technique is performed to remove noise
    blur = cv2.GaussianBlur(gray,(5,5),0)
    
    # Applied inversed thresh_binary 
    binary = cv2.threshold(blur, 100, 255,
                         cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    ## Applied dilation to increase the white region of the image
    kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)

In [14]:
# Create sort_contours() function to grab the contour of each digit from left to right
def sort_contours(cnts,reverse = False):
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    return cnts

cont, _  = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# creat a copy version "test_roi" of plat_image to draw bounding box
test_roi = plate_image.copy()

# Initialize a list which will be used to append charater image
crop_characters = []

# define standard width and height of character
digit_w, digit_h = 30, 60

for c in sort_contours(cont):
    (x, y, w, h) = cv2.boundingRect(c)
    ratio = h/w
    if 1<=ratio<=3.5: # Only select contour with defined ratio
        if h/plate_image.shape[0]>=0.5: # Select contour which has the height larger than 50% of the plate
            # Draw bounding box arroung digit number
            cv2.rectangle(test_roi, (x, y), (x + w, y + h), (0, 255,0), 2)

            # Sperate number and gibe prediction
            curr_num = thre_mor[y:y+h,x:x+w]
            curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
            _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            crop_characters.append(curr_num)
            
print("Detect {} letters...".format(len(crop_characters)))  

Detect 7 letters...


In [50]:
fig = plt.figure(figsize=(14,4))
grid = gridspec.GridSpec(ncols=len(crop_characters),nrows=1,figure=fig)

for i in range(len(crop_characters)):
    fig.add_subplot(grid[i])
    plt.axis(False)
    plt.imshow(crop_characters[i],cmap="gray")

![title](resources/pre_label.png)

In [2]:
# dataset_paths = glob.glob("resources/dataset_characters/**/*.jpg")

# cols=4
# rows=3
# fig = plt.figure(figsize=(10,8))
# plt.rcParams.update({"font.size":14})
# grid = gridspec.GridSpec(ncols=cols,nrows=rows,figure=fig)

# # create a random list of images will be displayed
# np.random.seed(45)
# rand = np.random.randint(0,len(dataset_paths),size=(cols*rows))

# # Plot example images
# for i in range(cols*rows):
#     fig.add_subplot(grid[i])
#     image = load_img(dataset_paths[rand[i]])
#     label = dataset_paths[rand[i]].split(os.path.sep)[-2]
#     plt.title('"{:s}"'.format(label))
#     plt.axis(False)
#     plt.imshow(image)

![title](resources/letras_4x3.png)

In [17]:
# Arange input data and corresponding labels
X=[]
labels=[]

for image_path in dataset_paths:
    label = image_path.split(os.path.sep)[-2]
    image=load_img(image_path,target_size=(80,80))
    image=img_to_array(image)

    X.append(image)
    labels.append(label)

X = np.array(X,dtype="float16")
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelEncoder()
lb.fit(labels)
labels = lb.transform(labels)
y = to_categorical(labels)

# save label file so we can use in another script
np.save('license_character_classes.npy', lb.classes_)

# split 10% of data as validation set
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.10, stratify=y, random_state=42)


# generate data augumentation method
image_gen = ImageDataGenerator(rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest"
                              )

print("[INFO] Find {:d} images with {:d} classes".format(len(X),len(set(labels))))

[INFO] Find 37623 images with 36 classes


In [18]:
pf_img_rec_model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(128, (3, 3), input_shape=(80, 80, 3)),
    tf.keras.layers.Conv2D(128, (3, 3),  activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.05),
    # The second convolution
    tf.keras.layers.Conv2D(256, (3, 3),  activation='relu'),
    tf.keras.layers.Conv2D(256, (3, 3),  activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 128 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(36, activation='softmax')
])

In [19]:
pf_img_rec_model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [52]:
# # #pf_img_rec_model_hist = pf_img_rec_model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=15, verbose=1)

![title](epochs_crop.JPG)

In [23]:
# save model architectur as json file
model_json = pf_img_rec_model.to_json()
with open("pf_img_rec_model_15e.json", "w") as json_file:  # modelin
    json_file.write(model_json)

pf_img_rec_model.save("pf_img_rec_model_weights_15e.h5")  # weights

In [24]:
# Load model architecture, weight and labels
json_file = open('pf_img_rec_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("pf_img_rec_model_weights.h5")
print("[INFO] Model loaded successfully...")

labels = LabelEncoder()
labels.classes_ = np.load('license_character_classes.npy')
print("[INFO] Labels loaded successfully...")

[INFO] Model loaded successfully...
[INFO] Labels loaded successfully...


In [25]:
# pre-processing input images and pedict with model
def predict_from_model(image,model,labels):
    image = cv2.resize(image,(80,80))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction

In [54]:
fig = plt.figure(figsize=(15,3))
cols = len(crop_characters)
grid = gridspec.GridSpec(ncols=cols,nrows=1,figure=fig)

final_string = ''
for i,character in enumerate(crop_characters):
    fig.add_subplot(grid[i])
    title = np.array2string(predict_from_model(character,model,labels))
    plt.title('{}'.format(title.strip("'[]"),fontsize=20))
    final_string+=title.strip("'[]")
    plt.axis(False)
    plt.imshow(character,cmap='gray')

print("Achieved result: ", final_string)
plt.savefig(final_string+'.png', dpi=300)

![title](2361BYR.png)